# 1. Пример импорта данных. Грузим данные

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# загрузка данных
path_to_dataset = '/Users/daniil/nn_ml_practice/insurance.csv'
data = pd.read_csv(path_to_dataset, sep=',')

# 2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (если у вас многоклассовая классификация, прочтите P.S.S. внизу).

Моя задача относится к регрессии, так как я пытаюсь предсказать медицинские расходы, которые представлены в виде непрерывных числовых значений и они не принимают дискретные значения, представляющие категории или классы.

# 3. Сделать предобработку данных:
Разделить выборку на тренировочную (train) и тестовую (test). Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test.
Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
Отнормировать численные переменные (StandardScaler, MinMaxScaler).
Закодировать категориальные признаки по одной из стратегий.
jupyter notebook

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

path_to_dataset = '/Users/daniil/nn_ml_practice/insurance.csv'
data = pd.read_csv(path_to_dataset, sep=',')

# проверка на наличие пропусков
print(data.isnull().sum())

# разделение данных на признаки и целевую переменную
X = data.drop('charges', axis=1)
y = data['charges']

# разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# определение числовых и категориальных признаков
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']

# пайплайн для числовых признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Заполнение пропусков средним значением
    ('scaler', StandardScaler())  # Нормализация StandardScaler
])

# пайплайн для категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # заполнение пропусков наиболее частым значением
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # OneHotEncoder для кодирования категориальных признаков
])

# комбинирование пайплайнов с помощью ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# применение преобразований к тренировочным и тестовым данным
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# проверка результирующих массивов
print(X_train.shape)
print(X_test.shape)

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64
(1070, 11)
(268, 11)


По выводу понятно, что пропусков у нас нет, заполнять их не нужно.
Тренировочная выборка содержит 1070 строк и 11 столбцов.
Тестовая выборка содержит 268 строк и 11 столбцов.
Мы нормализовали числовые переменные и закодировали категориальные признаки с помощью ColumnTransformer и Pipeline.

# 4. Обучить на тренировочном множестве:
Линейную модель (LogisticRegression, LinearRegression)
Деревянную модель (DecisionTreeClassifier, DecisionTreeRegressor) (тут советую попробовать разные глубины деревьев)
K-ближайших соседей (KNeighborsClassifier, KNeighborsRegressor) (тут тоже есть смысл попробовать разные k)
Случайный лес (RandomForestClassifier, RandomForestRegressor)

Мы рассмотрим следующие модели:

Линейная регрессия (LinearRegression)
Дерево решений для регрессии (DecisionTreeRegressor)
K-ближайших соседей для регрессии (KNeighborsRegressor)
Случайный лес для регрессии (RandomForestRegressor)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

path_to_dataset = '/Users/daniil/nn_ml_practice/insurance.csv'
data = pd.read_csv(path_to_dataset, sep=',')

# проверка пропусков
missing_values = data.isnull().sum()
print(missing_values)

# разделение данных на признаки (X) и целевую переменную (y)
X = data.drop(columns='charges')
y = data['charges']

# разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# определение числовых и категориальных признаков
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']

# создание трансформеров для числовых и категориальных признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# объединение трансформеров в единый препроцессор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# обучение линейной регрессии
lin_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

lin_reg.fit(X_train, y_train)
y_pred_lin_reg = lin_reg.predict(X_test)

# обучение дерева решений для регрессии
tree_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(max_depth=5))
])

tree_reg.fit(X_train, y_train)
y_pred_tree_reg = tree_reg.predict(X_test)

# обучение K-ближайших соседей для регрессии
knn_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor(n_neighbors=5))
])

knn_reg.fit(X_train, y_train)
y_pred_knn_reg = knn_reg.predict(X_test)

# обучение случайного леса для регрессии
forest_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42))
])

forest_reg.fit(X_train, y_train)
y_pred_forest_reg = forest_reg.predict(X_test)

# оценка моделей
models = {
    "Linear Regression": (y_pred_lin_reg, lin_reg),
    "Decision Tree (max_depth=5)": (y_pred_tree_reg, tree_reg),
    "K-Neighbors (k=5)": (y_pred_knn_reg, knn_reg),
    "Random Forest (max_depth=5)": (y_pred_forest_reg, forest_reg)
}

for model_name, (y_pred, model) in models.items():
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} - MSE: {mse}, R^2: {r2}")


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64
Linear Regression - MSE: 33596915.851361476, R^2: 0.7835929767120722
Decision Tree (max_depth=5) - MSE: 25857792.05855163, R^2: 0.8334428126395235
K-Neighbors (k=5) - MSE: 35984256.904628225, R^2: 0.7682154529775659
Random Forest (max_depth=5) - MSE: 19743528.653210577, R^2: 0.8728264735982286


# 5. Посчитайте метрики на train и test множествах:
Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
Для задачи регрессии -- MAE, RMSE, MAPE

In [3]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

ath_to_dataset = '/Users/daniil/nn_ml_practice/insurance.csv'
data = pd.read_csv(path_to_dataset, sep=',')

# Проверка пропусков
missing_values = data.isnull().sum()
print(missing_values)

# Разделение данных на признаки (X) и целевую переменную (y)
X = data.drop(columns='charges')
y = data['charges']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение числовых и категориальных признаков
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']

# Создание трансформеров для числовых и категориальных признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение трансформеров в единый препроцессор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Функция для вычисления метрик
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, mape

# Функция для вывода метрик
def print_metrics(model_name, y_train_true, y_train_pred, y_test_true, y_test_pred):
    train_mae, train_rmse, train_mape = calculate_metrics(y_train_true, y_train_pred)
    test_mae, test_rmse, test_mape = calculate_metrics(y_test_true, y_test_pred)
    print(f"{model_name} - Train MAE: {train_mae}, Train RMSE: {train_rmse}, Train MAPE: {train_mape}")
    print(f"{model_name} - Test MAE: {test_mae}, Test RMSE: {test_rmse}, Test MAPE: {test_mape}")

# Обучение и оценка линейной регрессии
lin_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

lin_reg.fit(X_train, y_train)
y_train_pred_lin_reg = lin_reg.predict(X_train)
y_test_pred_lin_reg = lin_reg.predict(X_test)

print_metrics("Linear Regression", y_train, y_train_pred_lin_reg, y_test, y_test_pred_lin_reg)

# Обучение и оценка дерева решений для регрессии
tree_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(max_depth=5))
])

tree_reg.fit(X_train, y_train)
y_train_pred_tree_reg = tree_reg.predict(X_train)
y_test_pred_tree_reg = tree_reg.predict(X_test)

print_metrics("Decision Tree (max_depth=5)", y_train, y_train_pred_tree_reg, y_test, y_test_pred_tree_reg)

# Обучение и оценка K-ближайших соседей для регрессии
knn_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor(n_neighbors=5))
])

knn_reg.fit(X_train, y_train)
y_train_pred_knn_reg = knn_reg.predict(X_train)
y_test_pred_knn_reg = knn_reg.predict(X_test)

print_metrics("K-Neighbors (k=5)", y_train, y_train_pred_knn_reg, y_test, y_test_pred_knn_reg)

# Обучение и оценка случайного леса для регрессии
forest_reg = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42))
])

forest_reg.fit(X_train, y_train)
y_train_pred_forest_reg = forest_reg.predict(X_train)
y_test_pred_forest_reg = forest_reg.predict(X_test)

print_metrics("Random Forest (max_depth=5)", y_train, y_train_pred_forest_reg, y_test, y_test_pred_forest_reg)


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64
Linear Regression - Train MAE: 4208.234572492226, Train RMSE: 6105.545160099848, Train MAPE: 42.20268508031658
Linear Regression - Test MAE: 4181.194473753652, Test RMSE: 5796.284659276274, Test MAPE: 46.88825597914694
Decision Tree (max_depth=5) - Train MAE: 2323.7044277482587, Train RMSE: 4142.694257687975, Train MAPE: 27.961164929868204
Decision Tree (max_depth=5) - Test MAE: 2911.1600498786056, Test RMSE: 5085.055757663983, Test MAPE: 34.55446483446007
K-Neighbors (k=5) - Train MAE: 2859.823881779439, Train RMSE: 4773.829774933797, Train MAPE: 29.21651246242549
K-Neighbors (k=5) - Test MAE: 3631.59117925, Test RMSE: 5998.687931925466, Test MAPE: 40.29677995026158
Random Forest (max_depth=5) - Train MAE: 2254.839625100894, Train RMSE: 4043.7311642356385, Train MAPE: 27.31552161757523
Random Forest (max_depth=5) - Test MAE: 2558.6914480028695, Test RMSE: 4443.369065608953, T

# 6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
Какая модель справилась лучше с поставленной задачей?
Имеет ли место переобучение?
Имеет ли место недообучение?
Как можно улучшить метрики моделей?

1) Какая модель справилась лучше с поставленной задачей?
Random Forest (max_depth=5) показала наилучшие результаты среди всех моделей:
Train MAE: 2254.84
Train RMSE: 4043.73
Train MAPE: 27.32
Test MAE: 2558.69
Test RMSE: 4443.37
Test MAPE: 31.47
2) Имеет ли место переобучение?
Decision Tree (max_depth=5): Есть признаки переобучения, так как ошибка на тестовой выборке (Test MAE: 2911.16, Test RMSE: 5085.06) значительно выше, чем на тренировочной выборке (Train MAE: 2323.70, Train RMSE: 4142.69).

K-Neighbors (k=5): Тоже наблюдается переобучение, так как метрики на тестовой выборке (Test MAE: 3631.59, Test RMSE: 5998.69) хуже, чем на тренировочной (Train MAE: 2859.82, Train RMSE: 4773.83).

Random Forest (max_depth=5): Переобучение менее выражено, метрики на тестовой выборке не сильно отличаются от метрик на тренировочной.
3) Имеет ли место недообучение?
Linear Regression: В некоторой степени да, поскольку метрики на тренировочной выборке (Train MAE: 4208.23, Train RMSE: 6105.55) и на тестовой выборке (Test MAE: 4181.19, Test RMSE: 5796.28) достаточно высоки, что указывает на недостаточную модельную сложность для улавливания паттернов в данных.
4) Как можно улучшить метрики моделей?
- Увеличение сложности моделей (Для дерева решений можно попробовать увеличить глубину дерева, для случайного леса можно увеличить количество деревьев или их глубину, для k-ближайших соседей можно попробовать разные значения k)
- Тюнинг гиперпараметров (провести поиск гиперпараметров (например, с использованием GridSearchCV или RandomizedSearchCV) для каждой модели, чтобы найти оптимальные параметры)
- Добавление новых признаков (Инженерия признаков может помочь улучшить модель, добавив новые релевантные признаки или преобразования текущих)
- Использование ансамблей (Можно попробовать более сложные ансамблевые методы, такие как Gradient Boosting Machines (GBM) или XGBoost)
- Улучшение предобработки данных (Проверить и улучшить текущую нормализацию и кодирование признаков)